In [1]:
pip install easyocr

  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 19.9 MB/s eta 0:00:00a 0:00:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 8.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 10.1 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
import easyocr

In [5]:
# Load the reader for Sinhala
reader = easyocr.Reader(['sinh'])

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1018)>

In [6]:
import pytesseract
from PIL import Image
from tqdm import tqdm
import pandas as pd

In [7]:
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

In [8]:
from datasets import load_dataset

dataset = load_dataset("Ransaka/sinhala_synthetic_ocr-large")

/Users/nevidujayatilleke/Documents/MSC - Research/OCR Comparative Analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 6969
    })
})


In [10]:
train_dataset = dataset["train"]

In [11]:
for i in range(5):
    print(train_dataset[i])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2600x128 at 0x10F11D010>, 'text': 'ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත් අතර ඔහු'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2600x128 at 0x10F1256D0>, 'text': 'මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ලගා කරලා දුන්නට දිනේශ් අයියට අනේක වාරයක්'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2600x128 at 0x10EBFB390>, 'text': 'සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩියෝව අහන්න පුළුවන්. හැබැයි'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2600x128 at 0x10F116650>, 'text': 'ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය සම්පුර්ණ වු විට මැතිවරණයක්'}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2600x128 at 0x10F116650>, 'text': 'කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලිනන් හා ඩෙනිම් ඇඳුම්, යට ඇඳුම් පරාසයක් පවතින අතර අවශ්ය'}


In [12]:
def perform_ocr(image):
    try:
        text = pytesseract.image_to_string(image.convert('RGB'), lang='sin')
        return text.strip()
    except Exception as e:
        print(f"Error during OCR: {e}")
        return None

In [13]:
results = []
for i in tqdm(range(len(dataset['train']))):
    item = dataset['train'][i]
    image = item['image']
    reference_text = item['text']
    generated_text = perform_ocr(image)
    results.append({'reference': reference_text, 'generated': generated_text})

100%|██████████| 6969/6969 [12:38<00:00,  9.19it/s]


In [14]:
df = pd.DataFrame(results)

In [17]:
df.head(20)

,reference,generated
0,ඔහු සතුව තිබී හෙරොයින් කිලෝග්රෑම් 1ක් සොයා ගත්...,ඔහු සනුව තිබී හෙරොයින්‌ කිලෝග්රෑම්‌ 1ක්‌ සොයා ...
1,"මේ තරම් මහන්සි වෙලා, ලස්සන අත්දැකීමක් අපිටත් ල...","මේ තරම්‌ මහන්සි වෙලා, ලස්සන අත්දැකීමක්‌ අපිටත්..."
2,සාමාන්යයෙන් කාර් බැටරියකින් මාස දෙකක් පමණ රේඩි...,සාමාන්යයෙන්‌ කාර්‌ බැටරියකින්‌ මාස දෙකක්‌ පමණ ...
3,ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...,ජනාධිපති මහින්ද රාජපක්ෂට තම වසර 4 ක නිල කාලය ස...
4,"කාර්යාලීය ඇඳුම්, හවස්වරුවේ සාද සඳහා ඇදුම්, ලින...","ය]1[1(81[:)8 £1/1(26), (1)£)(3£)(1/:1ව] 316 (3..."
5,නිවනට යන්න තෘශ්ණාවක් ඕනේ ඒක කුසලමූල තන්හාව.. ඒ...,නුවනට යන්න ඞෘ0%වාවක්‌ ඕනේ ඒ කූසලමූල නන්ඩාව.. ඒ...
6,", 4 , , , , . 0772104663, 0112845449 කොට්ටාව ක...",", 4 , , , , . 0772104603, 0112849410 කොට්ටාව ක..."
7,බුදු සරණයි පුතේ.... ක්ෂුද්‍ර ජීවී විද්‍යාව පිල...,බුදු සරණයි පුතේ.... ක්ෂුද්‍ර රීවී විද්‍යාව පිල...
8,හරීන්ද්‍ර මම කැමැති චරිතයක්. ඔබට සැප හා ජය. සච...,හරීන්ද්‍ර මම කැමැති චරිතයක්‌. ඔබට සැප හා ජය. ස...
9,මාධ්ය නිදහස නැ කියලා බෙරිහන් දෙන මහින්ද ප්රමුක...,මාධිය නිදහස නැ කියලා බෙරිහන්‌ දෙන මහින්ද ප්රමු...


In [18]:
df.to_csv("Tesseract_OCR_Results.csv", index = False)

In [19]:
from jiwer import wer, cer

In [29]:
agg_error_wer = 0
agg_error_cer = 0
for i in range(len(df)):
    record = df.iloc[i]
    try:
        error_wer = wer(record['reference'], record['generated'])
        error_cer = cer(record['reference'], record['generated'])
    except:
        print(record)
        print(i)
    agg_error_wer += error_wer
    agg_error_cer += error_cer

reference    None
generated        
Name: 2929, dtype: object
2929


In [30]:
print(f"Word Error Rate (WER): {agg_error_wer/ len(df)}")

Word Error Rate (WER): 0.33510160424609364


In [32]:
print(f"Character Error Rate (CER): {agg_error_cer/ len(df)}")

Character Error Rate (CER): 0.12540145511045728
